# toxic comments classification
## by Karin Brisker





### imports

In [10]:
# coding: utf-8
import datetime
import os
import torch.optim as optim
import torch.onnx
import random
import time
from torch.nn import DataParallel
import pandas as pd
import numpy as np
import re
import time
import torch
import torch.nn as nn
torch.backends.cudnn.enabled = False
import torch.nn.functional as F


from torch.autograd import Variable
from tqdm import tqdm
import string
import itertools
import re
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from string import punctuation
from nltk.stem import PorterStemmer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

hyper parameters

In [3]:
lr = 0.01
clip = 0.25 # gradient clipping
epochs = 20
batch_size = 64
seed = 1234 # random seed
wdecay = 5e-5
dropout = 0.5
max_norm = 3.0
kernel_num = 256
kernel_sizes = [3, 4, 5]
save_dir = './'

# Set the random seed manually for reproducibility.
torch.manual_seed(seed)
random.seed(seed)
np.random.seed(seed)
device = torch.device("cuda:2")
# device = "cpu"
print(device)

cuda:2


In [4]:
# load glove embeddings
def load_glove():
    embeddings_list = []
    words = []
    embeddings = {}
    f = open('./Data/glove.6B.100d.txt', encoding='utf-8')
    for line in f:
        values = line.split()
        word = values[0]
        words.append(word)
        value = torch.tensor(np.asarray(values[1:], dtype='float32'))
        embeddings[word] = value
        embeddings_list.append(value)
    embeddings["<pad>"] = torch.zeros(100)
    embeddings_list.append(torch.zeros(100))
    f.close()
    vocab_size = len(embeddings.keys())
    idx2word = dict(enumerate(embeddings.keys()))
    word2idx = dict(zip(idx2word.values(), idx2word.keys()))
    return embeddings_list, vocab_size, word2idx, idx2word, embeddings, words

clean data functions

In [5]:
def cleanData(text, remove_stops=False, stemming=False, lemmatization=False):
    text = text.lower().split()
    text = " ".join(text)
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+\-=]", " ", text)

    # Replace apostrophes with standard lexicons
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ", text)
    text = re.sub(r"\+", " ", text)
    text = re.sub(r"\-", " ", text)
    text = re.sub(r"\=", " ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"isn't", "is not", text)
    text = re.sub(r"aren't", "are not", text)
    text = re.sub(r"ain't", "am not", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"didn't", "did not", text)
    text = re.sub(r"shan't", "shall not", text)
    text = re.sub(r"haven't", "have not", text)
    text = re.sub(r"hadn't", "had not", text)
    text = re.sub(r"hasn't", "has not", text)
    text = re.sub(r"don't", "do not", text)
    text = re.sub(r"wasn't", "was not", text)
    text = re.sub(r"weren't", "were not", text)
    text = re.sub(r"doesn't", "does not", text)
    text = re.sub(r"'s", " is", text)
    text = re.sub(r"'re", " are", text)
    text = re.sub(r"'m", " am", text)
    text = re.sub(r"'d", " would", text)
    text = re.sub(r"'ll", " will", text)
    text = re.sub(r"--th", " ", text)
    text = re.sub('[()\"\t_\n.,:=!@#$%^&*-/[\]?|1234567890—]', ' ', text)

    # More cleaning
    text = re.sub(r"alot", "a lot", text)
    text = re.sub(r"what's", "", text)
    text = re.sub(r"What's", "", text)

    # Remove urls and emails
    text = re.sub(r'^https?:\/\/.*[\r\n]*', ' ', text, flags=re.MULTILINE)
    text = re.sub(r'[\w\.-]+@[\w\.-]+', ' ', text, flags=re.MULTILINE)

    # Replace words like sooooooo with so
    text = ''.join(''.join(s)[:2] for _, s in itertools.groupby(text))

    # Remove punctuation from text
    text = ''.join([c for c in text if c not in punctuation])

    # Remove all symbols
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+\-=]", " ", text)
    text = re.sub(r'[^A-Za-z\s]', r' ', text)
    text = re.sub(r'\n', r' ', text)
    text = re.sub('[()\"\t_\n.,:=!@#$%^&*-/[\]?|1234567890—]', ' ', text)

    text = re.sub(r'\d +', ' ', text)

    if remove_stops:
        text = " ".join([w for w in text.split() if w not in stop_words])

    if stemming:
        st = PorterStemmer()
        text = " ".join([st.stem(w) for w in text.split()])

    if lemmatization:
        wordnet_lemmatizer = WordNetLemmatizer()
        text = " ".join([wordnet_lemmatizer.lemmatize(w, pos='v') for w in text.split()])

    return text


special_character_removal = re.compile(r'[^a-z\d ]', re.IGNORECASE)
replace_numbers = re.compile(r'\d+', re.IGNORECASE)


def text_to_wordlist(text):
    text = text.lower().split()
    text = " ".join(text)
    text = special_character_removal.sub('', text)
    text = replace_numbers.sub(' ', text)
    return text

def new_len(x):
    return len(x.split())

In [6]:
stop_words = set(stopwords.words('english'))
train_df = pd.read_csv('./Data/train.csv')
test_df = pd.read_csv('./Data/test.csv')
print('Train shape: ', train_df.shape)
print('Test shape: ', test_df.shape)

# y values
labels_data_train = train_df.iloc[:, 2:].values
labels_data_test = test_df.iloc[:, 2:].values
labels_count = labels_data_train.sum(axis=0)

print(' ### loading glove ### ')

embeddings_list, vocab_size, word2idx, idx2word, embeddings, words = load_glove()

print(' ### finished loading glove ### ')

labels = list(train_df.columns[2:])

print(' ### cleaning data ### ')

train_df['comment_text'] = train_df['comment_text'].map(lambda x: cleanData(x, True, True, True))
train_df = train_df.dropna()
train_df = train_df[train_df['comment_text'].map(new_len) > 5]
list_sentences_train = train_df["comment_text"].values

print(' ### finished cleaning data ### ')

list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
y = train_df[list_classes].values
list_sentences_test = test_df["comment_text"].values

print(' ### text to wordlist ### ')
comments = []
for text in tqdm(list_sentences_train):
    comments.append(text_to_wordlist(text))

train_df['comment_text'] = pd.Series(comments)
train_df = train_df[train_df['comment_text'].map(type) == str]

test_comments = []
for text in list_sentences_test:
    test_comments.append(text_to_wordlist(text).strip())
print(' ### finished text to wordlist ### ')

Train shape:  (159571, 8)
Test shape:  (153164, 2)
 ### loading glove ### 
 ### finished loading glove ### 
 ### cleaning data ### 


  4%|▍         | 5369/137131 [00:00<00:02, 53684.04it/s]

 ### finished cleaning data ### 
 ### text to wordlist ### 


100%|██████████| 137131/137131 [00:02<00:00, 54339.40it/s]


 ### finished text to wordlist ### 


CNN model for classifing comments

In [7]:
class CNNModel(nn.Module):
    def __init__(self, n_filters, filter_sizes, output_dim,
                 dropout, device, batch_size, embeddings_vectors, vocab_size):
        super().__init__()
        self.batch_size = batch_size
        self.device = device
        D = 100
        C = 6
        Ci = 1
        Co = 256
        Ks = [3, 4, 5]
        self.embed = nn.Embedding(vocab_size, 100, padding_idx=400000)
        self.embed.weight.data.copy_(embeddings_vectors)
        self.embed.weight.requires_grad = False
        for p in self.embed.parameters():
            if p.requires_grad:
                p.requires_grad = False

        self.convs1 = nn.ModuleList([nn.Conv2d(Ci, Co, (K, D)) for K in Ks])
        self.fc_first = nn.Linear(len(filter_sizes) * n_filters, n_filters, bias=True)
        self.activation_function = nn.Tanh()
        self.device = torch.device('cuda:2')
        self.dropout = nn.Dropout(dropout)
        self.fc1 = nn.Linear(len(Ks) * Co, C)

    def forward(self, comments):
        # x = [self.prot_dict[p] for p in protein]
        # x = torch.stack(x).to(self.device) # N= 256, w=200, D= 400, C0=256
        comments = self.embed(comments)
        x = comments.unsqueeze(1)  # (N, Ci, W, D)
        x = [F.relu(conv(x)).squeeze(3) for conv in self.convs1]  # [(N, Co, W), ...]*len(Ks)
        x = [F.max_pool1d(i, i.size(2)).squeeze(2) for i in x]  # [(N, Co), ...]*len(Ks)
        x = torch.cat(x, 1)
        x = self.dropout(x)  # (N, len(Ks)*Co) # 256,768 // pooled 256,256
        logit = self.fc1(x)# (N, C)
        return logit


In [11]:
def get_batch(train_df):
    num_samples = train_df.shape[0]
    remainder = num_samples % batch_size
    for j in range(0, num_samples - remainder, batch_size):
        batch = train_df[j: j + batch_size]
        batch_comments = batch["comment_text"].values
        batch_labels = [torch.tensor(x) for x in batch[list_classes].values]
        batch_idx = []
        for i in range(len(batch_comments)):
            words_data = [word2idx[c] for c in batch_comments[i].split() if c in words]
            batch_idx.append(torch.LongTensor(words_data))
        protein_tensors_list = torch.nn.utils.rnn.pad_sequence(batch_idx, True, word2idx['<pad>'])
        yield protein_tensors_list, torch.stack(batch_labels)


def train(train_data, model):
    # Turn on training mode which enables dropout.
    model.train()
    count = 0
    total_loss, correct = 0., 0
    # shuffle data
    train_data = train_data.sample(frac=1).reset_index(drop=True)
    start_time = time.time()
    batches_range = int((len(train_data) - 1) / batch_size)
    my_generator = get_batch(train_data)
    for iter in tqdm(range(batches_range)):
        comments, targets = next(my_generator)
        output = model(comments.to(device))
        count += int(batch_size)
        loss = criterion(output.float(), Variable(targets).float().to(device))
        optimizer.zero_grad()
        loss.backward()
        # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        torch.nn.utils.clip_grad_norm_(params, clip)
        optimizer.step()
        total_loss += loss.detach()
    print('\n')
    print('-' * 89)
    print(
        '| time: {:5.2f}s | train loss {:5.8f} | lr {:2.5f}'.format((time.time() - start_time),
                                                                                             total_loss * 1.0 / count,
                                                                                             optimizer.param_groups[0][
                                                                                                 'lr']))
    print('-' * 89)
    return


def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


if __name__ == '__main__':
    embeddings_vectors = torch.stack(embeddings_list)
    model = CNNModel(kernel_num, kernel_sizes, 2, 0.5, device, batch_size, embeddings_vectors,
                      len(word2idx)).to(device)
    model = DataParallel(model, device_ids=[2, 1, 0, 3], output_device=2)  # run on all 4 gpus
    print(f'The model has {count_parameters(model):,} trainable parameters')
    criterion = nn.BCEWithLogitsLoss().to(device)
    params_model = filter(lambda p: p.requires_grad, model.parameters())
    params = list(params_model) + list(criterion.parameters())
    optimizer = optim.Adam(params, lr=lr, weight_decay=wdecay)

    # Loop over epochs.
    for epoch in range(1, epochs + 1):
        print('epoch: ' + str(epoch))
        epoch_start_time = time.time()
        train(train_df, model)
        print('-' * 89)
        print('epoch {:3d} | time: {:5.2f}s'.format(epoch, (time.time() - epoch_start_time)))
        print('-' * 89)
        if not os.path.isdir(save_dir):
            os.makedirs(save_dir)

        with open(os.path.join(save_dir, 'epoch_' + str(epoch) + '.pt'), 'wb') as f:
            torch.save(model.module.state_dict(), f)

    print('=' * 89)


  0%|          | 0/1841 [00:00<?, ?it/s]

The model has 509,446 trainable parameters
epoch: 1



  8%|▊         | 148/1841 [17:28<2:36:23,  5.54s/it]


 10%|▉         | 176/1841 [19:52<2:26:49,  5.29s/it]

KeyboardInterrupt: 